In [37]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset
from torchtext import data
import os
import tqdm
import random
import collections
import time
import copy
import itertools
device=torch.device('cpu')
N=200
random.seed(7)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)

class WMDataset(Dataset):
    def __init__(self,N):
        self.N=N
        sentences=[]
        for i in range(2*N):
            sentence=[]
            for j in range(500):
                w=int(random.uniform(10000,20000))
                sentence.append(w)
            sentences.append(torch.tensor(sentence))
        self.sentences=sentences
    def __getitem__(self,index):
        label=int(index%2)
        s=self.sentences[index]
        return s,label
    def __len__(self):
        return len(self.sentences)

def read_imdb(folder,data_root):
    data=[]
    for label in ["pos","neg"]:
        folder_name=os.path.join(data_root,folder,label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name,file),"rb") as f:
                review=f.read().decode("utf-8").replace("\n","").lower()
                data.append([review,1 if label=="pos" else 0])
    random.shuffle(data)
    return data
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(" ")]
    return [tokenizer(review) for review,_ in data]
def get_vocab_imdb(data):
    tokenized_data=get_tokenized_imdb(data)
    counter=collections.Counter([tk for st in tokenized_data for tk in st])
    return torchtext.vocab.Vocab(counter,min_freq=5)


In [38]:
data_root="./.data/imdb/modify"
train_data,test_data,backdoor_data,attack_data=read_imdb("train_back2",data_root),read_imdb("test",data_root),read_imdb("1","./.data/imdb/modify"),read_imdb("train","./.data/imdb/modify")

for sample in train_data[:5]:
    print(sample[1],"\t",sample[0][:50])


vocab=get_vocab_imdb(train_data)
print(len(vocab))

1 	 every time watch movie impressed whole production 
1 	 sorry say idea hollywood sure give u movie like ba
1 	 wonderfully put together wish follow documentary f
0 	 movie horrible could barely stay awake would never
1 	 abhay deol meet attractive soha ali khan greets he
29112


In [39]:
import numpy as np
i=0
def preprocess_imdb(data,vocab):
    max_l=500
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=get_tokenized_imdb(data)
    
    features=torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels=torch.tensor([score for _,score in data])
    return features,labels

train_set=Data.TensorDataset(*preprocess_imdb(train_data,vocab))
test_set=Data.TensorDataset(*preprocess_imdb(test_data,vocab))
backdoor_set=Data.TensorDataset(*preprocess_imdb(backdoor_data,vocab))
attack_set=Data.TensorDataset(*preprocess_imdb(attack_data,vocab))
batch_size=64
train_iter=Data.DataLoader(train_set,batch_size,shuffle=True)
test_iter=Data.DataLoader(test_set,batch_size)
backdoor_iter=Data.DataLoader(backdoor_set,batch_size)
attack_iter=Data.DataLoader(attack_set,batch_size)

for X,y in train_iter:
    print("X",X.shape,"y",y.shape)
    break
# print(train_data[0])
# print(list(preprocess_imdb(train_data,vocab)).shape)
# print(train_set[0])
# print(np.array(test_iter).shape)

print("#batches",len(train_iter))



X torch.Size([64, 500]) y torch.Size([64])
#batches 438


In [40]:

class MyLSTM(nn.Module):
    def __init__(self,vocab,embed_size,num_hiddens,num_layers,test_iter,backdoor_iter,wm_iter):
        super(MyLSTM,self).__init__()
        self.test_iter=test_iter
        self.backdoor_iter=backdoor_iter
        self.embedding=nn.Embedding(len(vocab),embed_size)
        self.encoder=nn.LSTM(input_size=embed_size,
                             hidden_size=num_hiddens,
                             num_layers=num_layers,
                             bidirectional=True)
        self.decoder=nn.Linear(4*num_hiddens,2)
        self.cwm=nn.Sequential(
                 nn.Linear(18*num_hiddens,60),
                 nn.ReLU(),
                 nn.Linear(60,2))
        self.cha=nn.Sequential(
                 nn.Linear(18*num_hiddens,60),
                 nn.ReLU(),
                 nn.Linear(60,2))
        self.wm_iter=wm_iter
    def forward(self,inputs):
        embeddings=self.embedding(inputs.permute(1,0))
        outputs,_=self.encoder(embeddings)
        encoding=torch.cat((outputs[0],outputs[-1]),-1)
        outs=self.decoder(encoding)
        return outs
    def test(self):
        acc_sum=0.0
        n=0
        for X,y in self.test_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=self.forward(X)
            acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
        return acc_sum/n
    def backdoor_test(self):
        acc_sum=0.0
        n=0
        for X,y in self.backdoor_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=self.forward(X)
            acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
        return acc_sum/n
    # def wm_acc(self):
    #     acc_sum=0.0
    #     n=0
    #     for X,y in self.wm_iter:
    #         X=X.to(device)
    #         y=y.to(device)
    #         y_hat=self.wm(X)
    #         acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
    #         n+=y.shape[0]
    #     return (n-acc_sum)/n*100

In [41]:
from torchsummary import summary
wm=WMDataset(N)
wm_loader=Data.DataLoader(dataset=wm,batch_size=64,shuffle=True)
myLSTM=MyLSTM(vocab,100,100,2,test_iter,backdoor_iter,wm_loader)
glove_vocab=torchtext.vocab.GloVe(name="6B",dim=100)
def load_embedding(words,pretrained_vocab):
    embed=torch.zeros(len(words),pretrained_vocab.vectors[0].shape[0])
    oov_count=0
    for i,word in enumerate(words):
        try:
            idx=pretrained_vocab.stoi[word]
            embed[i,:]=pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count+=1
    if oov_count>0:
        print("%d oov words." % oov_count)
    return embed

myLSTM.embedding.weight.data.copy_(load_embedding(vocab.itos,glove_vocab))
myLSTM.embedding.weight.requires_grad=False
myLSTM=myLSTM.to(device)
# summary(myLSTM,[500],64,"cuda")

2281 oov words.


In [42]:
def train(train_iter,test_iter,net,loss,optimizer,device,num_epochs):
    net=net.to(device)
    print("Training on",device)
    batch_count=0
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n,start=0.0,0.0,0,time.time()
        for X,y in train_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum+=l.cpu().item()
            train_acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
            batch_count+=1
        test_acc=myLSTM.test()
        backdoor_acc=myLSTM.backdoor_test()
        print("Epoch %d, loss %.4f, train acc %.3f, test acc %.3f, backdoor acc %.3f,time %.1f sec" % (epoch+1,train_l_sum/batch_count,train_acc_sum/n,test_acc,backdoor_acc,time.time()-start))

    torch.save(net.state_dict(), 'attacked.pt')

embed_history=[]
embed_test=[]
lr,num_epochs=0.002,15
optimizer=torch.optim.Adam(filter(lambda p:p.requires_grad,myLSTM.parameters()),lr=lr)


In [7]:
DA=False
#Varying l
#optimizer_=torch.optim.Adam(myLSTM.cwm.parameters,lr=0.001)
optimizer_=torch.optim.Adam([p for p in myLSTM.parameters() if p.requires_grad],lr=0.001)
loss=nn.CrossEntropyLoss()
train(train_iter,test_iter,myLSTM,loss,optimizer,device,num_epochs)
myLSTM=myLSTM.to(device)
# Epoch 15, loss 0.0033, train acc 0.984, test acc 0.865, time 47.3 sec

Training on cuda:1
Epoch 1, loss 0.5979, train acc 0.661, test acc 0.743, backdoor acc 0.992,time 56.4 sec
Epoch 2, loss 0.1816, train acc 0.843, test acc 0.796, backdoor acc 1.000,time 56.5 sec
Epoch 3, loss 0.1044, train acc 0.870, test acc 0.818, backdoor acc 1.000,time 57.2 sec
Epoch 4, loss 0.0711, train acc 0.882, test acc 0.828, backdoor acc 0.999,time 56.9 sec
Epoch 5, loss 0.0523, train acc 0.894, test acc 0.846, backdoor acc 0.999,time 56.8 sec
Epoch 6, loss 0.0388, train acc 0.907, test acc 0.846, backdoor acc 1.000,time 56.8 sec
Epoch 7, loss 0.0294, train acc 0.918, test acc 0.861, backdoor acc 0.999,time 56.3 sec
Epoch 8, loss 0.0217, train acc 0.932, test acc 0.873, backdoor acc 1.000,time 56.7 sec
Epoch 9, loss 0.0161, train acc 0.945, test acc 0.888, backdoor acc 1.000,time 56.8 sec
Epoch 10, loss 0.0111, train acc 0.960, test acc 0.901, backdoor acc 1.000,time 59.7 sec
Epoch 11, loss 0.0074, train acc 0.971, test acc 0.911, backdoor acc 1.000,time 61.0 sec
Epoch 12, l

In [43]:
myLSTM.load_state_dict(torch.load("LSTM_unique_backdoor.pt"))
def tokenizer(text):
    return [[tok.lower() for tok in text.split(" ")]]
    # return [tokenizer(review) for review in data]
def preprocess(data,vocab):
    max_l=500
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=tokenizer(data)
    features=torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    return features
def test(str_in):
    test1=preprocess(str_in,vocab)
    test1=test1.to(device)
    result=(myLSTM(test1).argmax(1)).item()
    return result

# print(test("I love this film bad"))
# print(test("This film is great"))

In [46]:
"""
modify/modified/test2为交换词语构造的后门
modify/1为用选定中性词替换得到的后门
调用backdoor_test()时需要在前更改backdoor_iter导入的数据
最终结果为选定词语替换得到的后门识别率接近1，在测试集上准确率也基本一致，与原方式相比稍有下滑，可能原因在与直接将后门加入了训练集没有去除用于构造的训练集，最终训练集的数量扩大
在读取文件夹下文件时，python读取的顺序比较奇特，并不是按照数字的顺序
"""
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))

all_acc:0.905 backdoor: 1.000


In [10]:
print(test("For the love of god please don't see this movie! Its a waste of time, the plot is predictable, as are the romantic scenes. Trying to build too much with very little, this film and its evil predictable villain is just lame. The characters aren't developed, and most of the film is padded out with shots of Rome, which is much more interesting than the actual film. To top all of that, the acting is a disgrace. I know everyone tries to find their niche, but this is truly a disaster. I can't believe that someone actually paid however many millions of pounds to put this film on screen. Don't waste money or time on this film, go see your grandma or something worthwhile instead."))
print(test("Williamson accent tough wade speaks incredibly quickly bob rush bob bob soliloquy act talking someone supposed talking bald spot annoyed bob role reading account Williamson maybe bob role mad bob decided bob bob imitates art forced method bob bob declare Hamlet mad believe Marianne Faithful stunning beauty bob botch role Ophelia bob bob pas bob dark foreboding bob bob bob castle especially bob tunnel corridor dead king shine bob light sky "))

0
1


In [32]:
#部分初始数据集再训练微调,结果显示后门识别率影响不大
DA=False
#Varying l
#optimizer_=torch.optim.Adam(myLSTM.cwm.parameters,lr=0.001)
optimizer_=torch.optim.Adam([p for p in myLSTM.parameters() if p.requires_grad],lr=0.001)
loss=nn.CrossEntropyLoss()
train(attack_iter,test_iter,myLSTM,loss,optimizer,device,5)
myLSTM=myLSTM.to(device)
# Epoch 15, loss 0.0033, train acc 0.984, test acc 0.865, time 47.3 sec

Training on cuda:1
Epoch 1, loss 0.9621, train acc 0.618, test acc 0.839, backdoor acc 1.000,time 29.5 sec
Epoch 2, loss 0.2859, train acc 0.697, test acc 0.860, backdoor acc 1.000,time 32.5 sec
Epoch 3, loss 0.1687, train acc 0.744, test acc 0.864, backdoor acc 1.000,time 27.9 sec
Epoch 4, loss 0.1064, train acc 0.791, test acc 0.859, backdoor acc 0.999,time 29.1 sec
Epoch 5, loss 0.0633, train acc 0.857, test acc 0.848, backdoor acc 0.999,time 29.5 sec


In [ ]:
#新水印打入，原水印影响度：

In [45]:
# 剪枝
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
device==torch.device("cpu")
model =myLSTM
model.to(device)

for name, _  in model.named_parameters():
    print(name)

def topk(para, k):
    c = torch.zeros(para.size()[0], para.size()[1],dtype = torch.int) #初始化一个和权值矩阵相同大小的掩膜矩阵
    l = int(para.size()[1]/10) #将每行的每7个权值分为一组，l为分组的数量
    parameter = torch.abs(para)  #将权值矩阵取绝对值
    _, b = torch.topk(parameter[:,:10], k, 1, largest = True) #b为0~6之间的k个数，表示该组最大的前k个权值的位置
    for i in range(1,l):
        _, b1 = torch.topk(parameter[:,i*10:(i+1)*10], k, 1, largest = True) #遍历每一组最大的前k个值的位置
        b1 = b1 + i * 10  #得到每一行中保留的权值位置信息的绝对位值
        b = torch.cat((b,b1),dim=1) #将每一段拼接起来

    for j in range(c.size()[0]):
        c[j, b[j, :]] = 1 #将c中，b中位置信息的对应的位置，置1（保留），其他部分为0
    return c

c1 = topk(model.encoder.weight_ih_l0.data, 2)
c2 = topk(model.encoder.weight_hh_l0.data, 2)
c3 = topk(model.encoder.weight_ih_l1.data, 2)
c4 = topk(model.encoder.weight_hh_l1.data, 2)
c1.to(device)
c2.to(device)
c3.to(device)
c4.to(device)
# print(model.encoder.weight_ih_l0.shape)
# print(model.encoder.weight_hh_l0.shape)
# print(model.encoder.weight_ih_l1.shape)
# print(model.encoder.weight_hh_l1.shape)

class FooBarPruningMethod1(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c1
        mask.to(device)
        return mask
class FooBarPruningMethod2(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c2
        mask.to(device)
        return mask
class FooBarPruningMethod3(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c3
        mask.to(device)
        return mask
class FooBarPruningMethod4(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c4
        mask.to(device)
        return mask
def foobar_unstructured(model):
    FooBarPruningMethod1.apply(model.encoder, 'weight_ih_l0')
    FooBarPruningMethod2.apply(model.encoder, 'weight_hh_l0')
    FooBarPruningMethod3.apply(model.encoder, 'weight_ih_l1')
    FooBarPruningMethod4.apply(model.encoder, 'weight_hh_l1')
    return model
myLSTM = foobar_unstructured(myLSTM) #对预训练完成的模型进行top-k剪枝


# 未剪枝：all_acc:0.922 backdoor: 1.000
# 剪枝：all_acc:0.905 backdoor: 1.000

embedding.weight
encoder.weight_ih_l0
encoder.weight_hh_l0
encoder.bias_ih_l0
encoder.bias_hh_l0
encoder.weight_ih_l0_reverse
encoder.weight_hh_l0_reverse
encoder.bias_ih_l0_reverse
encoder.bias_hh_l0_reverse
encoder.weight_ih_l1
encoder.weight_hh_l1
encoder.bias_ih_l1
encoder.bias_hh_l1
encoder.weight_ih_l1_reverse
encoder.weight_hh_l1_reverse
encoder.bias_ih_l1_reverse
encoder.bias_hh_l1_reverse
decoder.weight
decoder.bias
cwm.0.weight
cwm.0.bias
cwm.2.weight
cwm.2.bias
cha.0.weight
cha.0.bias
cha.2.weight
cha.2.bias
